# ELUC Predictor

In [ ]:
# %pip install zarr
# %pip install regionmask
# %pip install xgboost
# %pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import regionmask
import xarray as xr

In [ ]:
dataset = xr.open_zarr("processed/merged_aggregated_dataset_1850_2022.zarr.zip", consolidated=True)

In [ ]:
dataset = dataset.stack(latlon=('lat', 'lon'))

In [ ]:
regionmask.__version__

In [ ]:
# Train on countries 0-80, and test on the rest, and test on the last 15 years of data
country_mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.mask(dataset)
dataset = dataset.assign_coords(country=country_mask)

In [ ]:
dataset

In [ ]:
countries = regionmask.defined_regions.natural_earth_v5_0_0.countries_110
countries

In [ ]:
dataset.country

In [ ]:
dataset.groupby(dataset.country).count()

In [ ]:
np.unique(dataset.time)

## Remove NaN ELUC_diff

In [ ]:
# Create a boolean mask based on NaN values for ELUC_diff
mask = dataset['ELUC_diff'].isnull()

In [ ]:
# Filter the dataset based on the mask
dataset = dataset.where(~mask, drop=True)

## Train and test sets
Create a dataset for a single country to keep it small and easy to train on  
Country: United Kingdom  
Train on data between 1850 and 2007  
Test on data after 2007 (i.e. 2008 to 2021)

In [ ]:
countries_df = countries.to_dataframe()
countries_df

In [ ]:
countries_df[countries_df.names == 'United Kingdom']

In [ ]:
# train_da = dataset.where(dataset.country <= 80, drop=True).where(dataset.time <= 2007, drop=True)
# test_da = dataset.where(dataset.country > 80,  drop=True).where(dataset.time > 2007, drop=True)
train_da = dataset.where(dataset.country == 143, drop=True).where(dataset.time <= 2007, drop=True)
test_da = dataset.where(dataset.country == 143,  drop=True).where(dataset.time > 2007, drop=True)

In [ ]:
train_da

In [ ]:
test_da

In [ ]:
features = list(train_da.data_vars)
# This is the target
features.remove('ELUC_diff')
# Do not use:
features.remove('cell_area')
features.remove('cell_area_diff')

In [ ]:
features

In [ ]:
# Extract features and target variable from the training dataset
X_train = train_da[features]
y_train = train_da['ELUC_diff']

In [ ]:
# Extract features and target variable from the test dataset
X_test = test_da[features]
y_test = test_da['ELUC_diff']

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
np.isnan(y_test.values)

## XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
X_train = np.column_stack([X_train[feature].values for feature in features])

In [ ]:
model = xgb.XGBRegressor()

In [ ]:
%%time
model.fit(X_train, y_train)

In [ ]:
X_test = np.column_stack([X_test[feature].values for feature in features])

In [ ]:
preds = model.predict(X_test)

In [ ]:
preds

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(y_test, preds, squared=True)
rmse = mean_squared_error(y_test, preds, squared=False)
mae = mean_absolute_error(y_test, preds)
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

In [ ]:
preds.shape